## Setup

In [5]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=6)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [6]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_time

## Part a

In [7]:
# Imports
from math import prod

In [51]:
# Functions
def solve_a(input_data: str) -> int:
    """Solve part A of the puzzle.

    We count the number of valid t values for each (max_time, distance_goal) pair.
    Here, t is our time spent accelerating, thus our speed equals t
    Therefore, our distance = speed * time_remaining = t * (max_time - t), which is valid when > distance_goal
    """
    return prod(
        sum(1 if t * (max_time - t) > distance_goal else 0 for t in range(1, max_time))
        for max_time, distance_goal in [
            *zip(*[[*map(int, line.split()[1:])] for line in input_data.splitlines()], strict=True)
        ]
    )

In [52]:
# Correctness check
assert str(solve_a(example.input_data)) == example.answer_a

In [60]:
# Performance check
time_a = check_time(solve_a, input_data, number=500, repeat_times=10, unit="us")
print(f"Part A solution time: {time_a:.2f} μs")

Part A solution time: 12.52 μs


In [16]:
# Submit answer
puzzle.answer_a = solve_a(input_data)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


## Part b

### Brute force approach
Using the same approach as part a will take some time but should work.

In [10]:
# Functions
def solve_b_brute_force(input_data: str) -> int:
    """Solve part B of the puzzle by brute forcing the distance calculation for each t."""
    max_time, distance_goal = [int(line.split(":")[1].replace(" ", "")) for line in input_data.splitlines()]
    return sum(1 if t * (max_time - t) > distance_goal else 0 for t in range(1, max_time))

In [11]:
# Correctness check
assert str(solve_b_brute_force(example.input_data)) == example.answer_b

In [50]:
# Performance check
time_b_brute_force = check_time(solve_b_brute_force, input_data, number=3, repeat_times=3, unit="s")
print(f"Brute force approach takes {time_b_brute_force:.2f} s")

Brute force approach takes 2.55 s


### Binomial approach
The distance traveled is t * (max_time - t). This is a quadratic equation, which can be rearranged to -t^2 + max_time * t - distance_goal = 0

We can use the quadratic formula to find the roots of this equation, and determine the range of t values that satisfy the equation, i.e. all integers between the two roots.

In [14]:
# Imports
from math import sqrt

In [46]:
# Functions
def find_valid_t_quadratic(max_time: int, distance_goal: int) -> int:
    """We want to find the roots for t in the equation -t^2 + max_time * t - distance_goal = 0.

    The amount of valid t values is then amount of integers between the two roots.
    """
    a = -1
    b = max_time
    c = -distance_goal

    discriminant = b**2 - 4 * a * c
    root1 = (-b + sqrt(discriminant)) / (2 * a)
    root2 = (-b - sqrt(discriminant)) / (2 * a)
    return int(root2 // 1 - root1 // 1) - (1 if root2 % 1 == 0 else 0)


def solve_b_quadratic(input_data: str) -> int:
    """Solve part B of the puzzle using the quadratic formula."""
    max_time, distance_goal = [int(line.split(":")[1].replace(" ", "")) for line in input_data.splitlines()]
    return find_valid_t_quadratic(max_time, distance_goal)

In [15]:
# Correctness check
assert str(solve_b_quadratic(example.input_data)) == example.answer_b

In [65]:
# Performance check
time_b_quadratic = check_time(solve_b_quadratic, input_data, number=500, repeat_times=10, unit="us")
print(
    f"Quadratic formula approach takes {time_b_quadratic:.2f} μs"
    f", which is {1_000_000 * time_b_brute_force / time_b_quadratic:,.0f} x faster than brute force."
)

Quadratic formula approach takes 1.38 μs, which is 1,846,536 x faster than brute force.


That's a huge speedup, I don't think i've even seen an improvement of a factor over a million in a single AoC optimization before!

In [ ]:
# Submit answer
puzzle.answer_b = solve_b_quadratic(input_data)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 6! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


### Bonus: solving part A using the quadratic formula
We can also solve part A using the quadratic formula by finding the maximum distance traveled for each max. Let's see if that speeds things up.

In [66]:
# Functions
def solve_a_quadratic(input_data: str) -> int:
    """Solve part A of the puzzle using the quadratic formula."""
    return prod(
        find_valid_t_quadratic(max_time, distance_goal)
        for max_time, distance_goal in [
            *zip(*[[*map(int, line.split()[1:])] for line in input_data.splitlines()], strict=True)
        ]
    )

In [68]:
# Correctness check
assert str(solve_a_quadratic(example.input_data)) == example.answer_a

In [98]:
# Performance check
time_a_quadratic = check_time(solve_a_quadratic, input_data, number=500, repeat_times=10, unit="us")
print(
    f"Quadratic formula approach for part A takes {time_a_quadratic:.2f} μs"
    f", which is {time_a / time_a_quadratic:.2f} x faster than the original implementation."
)

Quadratic formula approach for part A takes 3.68 μs, which is 3.40 x faster than the original implementation.
